### Packages

In [ ]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Parametres

### function system

In [5]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

### function for 2d map

In [6]:
function check_fp_point(fp, point)
"""
    Проверка на совпадение точки с одним из состояний равновесия.
    
    Алгоритм
        - Находится кол-во состояний равновесия, задаётся некоторое ϵ.
        - Проходимся по всем состояниям равновесия и сравниваем их с точкой с помощью следующего неравенства:
            abs( (E_fp, X_fp, u_fp, Y_fp) - abs( (E_p, X_p, u_p, Y_p) ) ) < ϵ
            то есть, если расстояние между двумя точками по модулю строго меньше, чем ϵ
        - Если неравенство выполняется, то сохраняем индекс, который соответствует этому состоянию равновесия
        - Иначе сохраняем индекс -1
        - По выполнению цикла ищем максимум из всех индексов.
"""
    
    len = length(fp)
    ϵ = [0.001, 0.001, 0.001, 0.001]
    res = zeros(len)
    
    for index in range(1, len, step = 1)
    
        if ( abs.(fp[index] - abs.(point)) ) < ϵ
            res[index] = index
        else
            res[index] = -1

        end
        
    end
    return convert(Int64, maximum(res))
end

function check_eigs_SLE(λλ, eigs)
"""
    Проверка на совпадение SLE (Спектр Ляпуновских показателей) и собственных чисел состояния равновесия
    
    Алгоритм
        - Задаётся ϵ вектор.
        - Если разница между собственными числами состояния равновесия и SLE меньше заданного ϵ вектора, то вернуть true
        - иначе вернуть false
"""
    ϵ = [0.01, 0.01, 0.01, 0.01]
    dif = abs.( sort( real(eigs) ) + (-1) * sort( λλ ) )
    
    if dif < ϵ
        return true # bad
    else
        return false # good
    end
end

function check_start(u0_started = SA[3.3403239669724387, 0.1, 0.1, 0.03677942307955071])

    if flag_start == true
                global u0_loc = u0_started
                flag_start == false
    end
    
end
function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

separate (generic function with 1 method)

### 2d map

In [9]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5

tfinal = 500.0
initial_condition = SA[0.0, 0.2, 0.1, 0.1]

I0_start =  -1.65
I0_finish = -1.5
I0_step = 0.01

U0_start =  0.35
U0_finish = 0.32
U0_step = -0.002

range_U0 = range(U0_start, U0_finish, step = U0_step)
range_I0 = range(I0_start, I0_finish, step = I0_step)

array_u0 = zeros(length(range_U0), length(range_I0), 4 )
array_cycles = zeros(length(range_U0), length(range_I0));

In [11]:
for index in range( 1, length( range_U0 ), step = 1 )
        
    U0 = range_U0[index]
    parametres = SA[τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_start]
    
    println("parametres: $parametres");flush(stdout)
    println("initial condition: $initial_condition");flush(stdout)
    ds = ContinuousDynamicalSystem( model, initial_condition, parametres )
    tr = trajectory( ds, tfinal, Ttr = tfinal; Δt = 0.001,
                        diffeq = ( alg = RK4(),  adaptive = false, dense = false, dt = 0.001 ))
    initial_condition = tr[end, :]
    println("last point from trajectory: $initial_condition");flush(stdout)
    random_point = tr[300000, :]
    println("random point from trajectory: $random_point");flush(stdout)
    if initial_condition - random_point <= [0.001, 0.001, 0.001, 0.001]
        cycle = false
    else
        cycle = true
    end
    
    
end

parametres: [0.013, 1.5, 0.15, 3.07, 0.348, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.346, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.344, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.342, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.34, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.338, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.336, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.334, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.332, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.33, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.328, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15, 3.07, 0.326, 0.305, 1.8, 0.4375, 0.9, 0.5, -1.65]
parametres: [0.013, 1.5, 0.15,

In [10]:
for (index_i, U0_lc) in enumerate(range_U0)
    
    if index_i == 1
        global u0_lc = SA[0.0, 0.2, 0.1, 0.1]
    end
            
    output(index_i, 1, U0_lc, I0_start, u0_lc)
    
    p_loc = SA[τ, α, τ_D, J, U0_lc, ΔU0, τ_y, β, xthr, ythr, I0_start]
    ds_loc = ContinuousDynamicalSystem(model, u0_lc, p_loc)

    tr_loc = trajectory(ds_loc, tfinal, Ttr = tfinal; Δt = 0.001, diffeq = (alg = RK4(),  adaptive = false, dense = false, dt = 0.001))
    u0_lc = tr_loc[end, :]
    array_u0[1, index_i, :] = u0_lc
    
    println("last point: $u0_lc");flush(stdout)
    
    random_dot = tr_loc[300000, :]
    println("random point: $random_dot")
    
    if abs.(u0_lc - random_dot) <= [0.001, 0.001, 0.001, 0.001]
        color = 0
    else
        color = 1
    end
    
    array_cycles[1, index_i] = color
    println("distance between last point and random point: ",u0_lc - random_dot )
    println("Cycles: (1 - true, 0 - false): $(array_cycles[1, index_i]);")
    separate()
end

index i: 1
index j: 1
U0: 0.35;
I0: -1.65;
u0: [0.0, 0.2, 0.1, 0.1];
last point: [1.4532726406060006, 0.6128188288248325, 0.8216678336611694, 0.00848216076958664]
random point: [5.053001361489919, 0.7382969896887166, 0.7769133749430952, 0.009433415783485957]
distance between last point and random point: [-3.5997287208839186, -0.12547816086388408, 0.04475445871807415, -0.0009512550138993173]
Cycles: (1 - true, 0 - false): 1.0;
 --------------------- 
  
index i: 2
index j: 1
U0: 0.348;
I0: -1.65;
u0: [1.4532726406060006, 0.6128188288248325, 0.8216678336611694, 0.00848216076958664];
last point: [12.481696002400767, 0.677485545899536, 0.7796398339025876, 0.009997539589038157]
random point: [28.173584707403965, 0.3821066988666861, 0.8309066201765799, 0.009788871381415877]
distance between last point and random point: [-15.691888705003198, 0.29537884703284994, -0.05126678627399228, 0.00020866820762227914]
Cycles: (1 - true, 0 - false): 1.0;
 --------------------- 
  
index i: 3
index j: 1
U

In [ ]:
range_I0_new = range(I0_start, I0_finish, step = I0_step)

for (index_i, U0_lc) in enumerate(range_U0)
    

        for (index_j, I0_lc) in enumerate(range_I0_new)
            
            if index_j == 1
                continue
            end
            if index_j == 2
                global u0_lc = array_u0[index_j - 1, index_i, :]
            end
            
            output(index_i, index_j, U0_lc, I0_lc, u0_lc)
            p_loc = SA[τ, α, τ_D, J, U0_lc, ΔU0, τ_y, β, xthr, ythr, I0_lc]
            println("p_loc: $p_loc");flush(stdout)
            ds_loc = ContinuousDynamicalSystem(model, u0_lc, p_loc)

            tr_loc = trajectory(ds_loc, tspan[2], Ttr = tspan[2]; Δt = 0.001, diffeq = (alg = RK4(),  adaptive = false, dense = false, dt = 0.001))
            u0_lc = tr_loc[end, :]
            println("last point: $u0_lc");flush(stdout)
            array_u0[index_j, index_i, :] = u0_lc
        
            random_dot = tr_loc[300000, :]
            println("random point: $random_dot")
        
            if abs.(u0_lc - random_dot) <= [0.001, 0.001, 0.001, 0.001]
                color = 0
            else
                color = 1
            end
        
            array_cycles[index_j, index_i] = color
    
            println("distance between last point and random point: ",u0_lc - random_dot )
            println("Cycles: (1 - true, 0 - false): $(array_cycles[index_j, index_i]);")
            separate()
        end

    
end

In [ ]:
fig, ax, hm = heatmap(range_I0, range_U0, array_cycles[:, :], colormap = :seismic)
Colorbar(fig[:, end+1], hm)

fig